In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Orders_app").getOrCreate()
spark

In [2]:
# Loading Data into RDD
orders_rdd = spark.sparkContext.textFile("hdfs://localhost:9000/user/tkm/data/orders.csv")

In [4]:
orders_rdd.take(5)

[',order_id,date,customer_id,status',
 '0,1,09/11/2020,3371,CLOSED',
 '1,2,12/01/2020,3902,PENDING',
 '2,3,12/09/2020,3009,COMPLETE',
 '3,4,22/08/2020,3023,PENDING']

In [6]:
# removing header
header = orders_rdd.first()
header

',order_id,date,customer_id,status'

In [8]:
data_rdd = orders_rdd.filter(lambda x : x != header)
data_rdd.take(5)

['0,1,09/11/2020,3371,CLOSED',
 '1,2,12/01/2020,3902,PENDING',
 '2,3,12/09/2020,3009,COMPLETE',
 '3,4,22/08/2020,3023,PENDING',
 '4,5,17/04/2020,4315,PENDING']

Orders in each Category

In [11]:
status_rdd = data_rdd.map(lambda x : (x.split(",")[4] , 1))

In [13]:
status_rdd.reduceByKey(lambda x,y : x+y).take(3)

[('CLOSED', 341), ('COMPLETE', 341), ('PENDING', 318)]

Find the Premium Customers

In [15]:
customer_rdd = data_rdd.map(lambda x : (x.split(",")[3] , 1))

In [20]:
customer_order_rdd = customer_rdd.reduceByKey(lambda x,y : x+y )

In [22]:
customer_order_rdd.sortBy(lambda x : x[1] , False).take(10)

[('4704', 5),
 ('4183', 5),
 ('3640', 4),
 ('3038', 4),
 ('3952', 4),
 ('4521', 4),
 ('3401', 4),
 ('3552', 3),
 ('4463', 3),
 ('4422', 3)]

Distinct Customers who placed atleast 1 order

In [23]:
data_rdd.map(lambda x : (x.split(",")[3] , 1)).distinct().count()

786

Customers having maximum number of CLOSED orders

In [25]:
closed_rdd = data_rdd.filter(lambda x : (x.split(",")[4] == "CLOSED"))

In [29]:
closed_order_rdd = closed_rdd.map(lambda x : (x.split(",")[3] , 1))

In [33]:
closed_order_customer_rdd = closed_order_rdd.reduceByKey(lambda x,y : x+y )

In [34]:
closed_order_customer_rdd.sortBy(lambda x : x[1] , False).first()

('4341', 3)